### Imports

In [18]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

### Recolha de dados

In [ ]:
# 64 patients with breast cancer and 52 healthy controls, 116 in total
# 1 - paciente sem cancro, 2 - paciente com cancro

file = pd.read_excel("../Data/dataR2.XLSX")
DataSet = pd.DataFrame(data=file.dropna()) #Remove NaN because of initial empty lines

# Find the indexes where the differente class samples are located
ixHealthy = np.where(DataSet['Classification'] == 1)
ixWithCancer = np.where(DataSet['Classification'] == 2)

# Definir classes
Classes = ["healthy","withCancer"]


DataSet['Classification'].iloc[ixHealthy] = Classes[0] 
DataSet['Classification'].iloc[ixWithCancer] = Classes[1] 

#for w in DataSet.columns:
#    for i in DataSet.columns: 
#        if w == 'Classification' or i == 'Classification' or w == i:
#            continue
#        fig = px.scatter_3d(x = DataSet.index, y = DataSet[w], z = DataSet[i], color = DataSet['Classification'], labels=dict(x = "N", y = w, z = i , color = "Classification"))
#        #fig = px.scatter_3d(x = CorkStopp["N.1"], y = CorkStopp["ARM"],z = CorkStopp["PRM"],color = CorkStopp['C'], labels=dict(x = "N", y = "ARM", color = "Cork Stoppers"))
#
#        fig.update_traces(marker_size=10)
#        fig.show()

# Partição
x = DataSet.iloc[:, :-1] # todas menos a última coluna

y = DataSet.iloc[:, -1] # apenas a última coluna


x_train, x_testInicial, y_train, y_testInicial = train_test_split( # Partições entre train e test
    DataSet, y,
    test_size=0.2,       # 20% dos dados para teste
    random_state=67,     # garante reprodutibilidade
    shuffle=True,
    stratify = y           # mantém a proporção das classes TODO: se calhar tirar depois
)


x_val, x_test, y_val, y_test = train_test_split( # Partições entre test e validation
    x_testInicial, y_testInicial,
    test_size=0.5,
    random_state=67,
    stratify=y_testInicial  # mantém a proporção das classes TODO: se calhar tirar depois
)


# Kruskall

X= x.to_numpy()
X=X.astype(float)

fnames=x.columns

Hs={}
for i in range(np.shape(X)[1]):
    st=stats.kruskal(X[ixHealthy,i].flatten(),X[ixWithCancer,i].flatten())
    Hs[fnames[i]]=st.statistic


Hs = sorted(Hs.items(), key=lambda x: x[1],reverse=True)  

print("Ranked features")

for f in Hs:
    print(f[0]+"-->"+str(f[1]))

#print(X)



Ranked features
Glucose-->23.91957997825514
Resistin-->9.699241863905343
HOMA-->8.887111686390483
Insulin-->4.931023337417101
BMI-->1.6374345259560847
Age-->0.5053505863773902
MCP.1-->0.4512315968813051
Adiponectin-->0.08986720937572039
Leptin-->0.004437869822481844


C:\Users\pedro\AppData\Local\Temp\ipykernel_13176\3766315131.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataSet['Classification'].iloc[ixHealthy] = Classes[0]
